In [1]:
import os
import sys
import torch
import time
import numpy as np
np.set_printoptions(suppress=True)
import cv2

In [2]:
sys.path.append("../0.mc_utils/")

In [3]:
from cameras.single3d_mats import qslerp,rot6d_to_rotmat,rodrigues2quat,quat2rodrigues
from openpose.op_formats import NPMapOP25bToSMPL24
from openpose.op_utils import draw_kp2d_to_image,draw_reided_bboxes
from mqtt_player.unity3d_mqtt_player import Unity3DMqttPlayer,MQTTPLAYER_CFG
from common.unity_visualizer import export_Amatrix,play_kp2ds_animation
from common.torch_x2ds_normalizer import XBBoxNormalier
from mchmr2.hmr_cfg import HMR_ENCODER
from mchmr2.hmr_encoder import HMREncoder
from common.tools_cv import draw_kp

from pipeline.pipeline_utils import ComputeSMPL24CameraTrajectory
from pipeline.human_instance import SMPL24HumanInstance

In [5]:
op25b_x2ds0 = np.load("C:/Users/mc/Desktop/surfing/pose1/2dpose.npz",allow_pickle=True)["op25b"].item()
video0_fname = "C:/Users/mc/Desktop/surfing/write_C0001_create.mp4"

##sub view
op25b_x2ds1 = np.load("C:/Users/mc/Desktop/surfing/pose2/2dpose.npz",allow_pickle=True)["op25b"].item()
video1_fname = "C:/Users/mc/Desktop/surfing/write_C0002_create.mp4"

In [6]:
mview_cfg = HMR_ENCODER.clone()
mview_cfg.hmr.model = "../12.models/mchmr2/20200510-hmrzeofusion-x2b5f2048s5-vposer0420-30.25.pth"
mview_cfg.hmr.in_features=96
mview_cfg.hmr.out_features=6+32+5
mview_cfg.hmr.blocks=5
mview_cfg.freeze()
# print(mview_cfg)

In [7]:
map_op25b_to_smpl24 = NPMapOP25bToSMPL24()
compute_traj = ComputeSMPL24CameraTrajectory(K=[1145,1145,1920/2,1080/2],traj_offset=[0.0,-1.2,0.0])
human0 = SMPL24HumanInstance()
human1 = SMPL24HumanInstance()
hmr_fusioner = HMREncoder(mview_cfg)

>>>Loaded HMR from:../12.models/mchmr2/20200510-hmrzeofusion-x2b5f2048s5-vposer0420-30.25.pth
>>>Loaded VPoser from:../12.models/mchmr2/20200420-vposer1024-lt32b3.pth


In [8]:
# triangulated_traj = np.load(f'{os.path.dirname(video0_fname)}/trajs.npy').reshape((-1,1,3))

In [10]:
mview_x2ds_dict = {}
A_list = []
r6d_list = []
traj_list = []
human0.reset()
human1.reset()
iss = []
for i in range(10000):
    if i in op25b_x2ds0 and i in op25b_x2ds1:
        ##step1.load x2ds
        print('\r%i'%(i),end='')
        x2ds0 = map_op25b_to_smpl24(op25b_x2ds0[i])[...,:2] #1x24x3
        x2ds1 = map_op25b_to_smpl24(op25b_x2ds1[i])[...,:2] #1x24x3
        mview_x2ds_dict[i] = [x2ds0[:1],x2ds1[:1]]
        
        ##step2.smoother
        x2ds0 = human0.update_x2ds(i,torch.from_numpy(x2ds0).to(torch.float32)[0]).unsqueeze(0)
        x2ds1 = human1.update_x2ds(i,torch.from_numpy(x2ds1).to(torch.float32)[0]).unsqueeze(0)
        
        ##step3.fusioner
        x = torch.stack([x2ds0,x2ds1],dim=1)
        ret = hmr_fusioner.forward_hmr(x)
        ret["root_r6d"][0] = human0.root_filter(i,ret["root_r6d"][0])
        ret["pose_r6d"][0] = human0.pose_filter(i,ret["pose_r6d"][0])
        r6d_list.append(torch.cat([ret["root_r6d"][0].unsqueeze(0),ret["pose_r6d"][0]],dim=0))
        
        A = hmr_fusioner.forward_smpl(ret["root_r6d"],ret["pose_r6d"]).cpu().numpy()
        
#         trajs = triangulated_traj[i:i+1]*np.array([[-1,1,1]])+np.array([[0.0,-1.2,0.0]])
#         trajs[0,[0,2]] = human0.traj_filter(i,trajs[0,[0,2]])
#         traj_list.append(trajs[0])
        
#         A[0,:,:3,-1] += trajs[0].reshape(1,3)  #update traj in A
        A = np.matmul(compute_traj.camera_to_world_R,A)
        A_list.append(A)
np_A = np.array(A_list)
path= 'C:/Users/mc/Desktop/surfing'
np.save(path+'/3dpose.npy',np_A)
np_A = np_A.reshape((-1,384))
fps = np.ones((1,384))*30
np_A = np.concatenate([fps,np_A])
np.savetxt(path+'/3dpose.txt',np_A)

1482

In [22]:
Amats = A.copy()
Amats = Amats.squeeze()

# trajs = np.load(f'{os.path.dirname(video0_fname)}/trajs.npy').reshape((-1,1,3))
# trajs[:,:,1] += 1.05
# N = min(Amats.shape[0],trajs.shape[0])
# print(Amats[:N,:,0:3,3].shape, trajs[:N].shape)
# Amats[:N,:,0:3,3]+=trajs[:N]

Amats = Amats.squeeze().reshape((-1,384))
export_Amatrix(f'{video0_fname[:-4]}_new_fusion.txt', Amats, fps=30)

In [54]:
rs = np.load(f'{os.path.dirname(video0_fname)}/rs.npy')
rs = torch.tensor(rs).type(torch.float32)
body_quats = body_quats_.cpu()
R = quat2rodrigues(body_quats.reshape(-1,4)).reshape(-1,24,3,3)

N = min(R.shape[0],rs.shape[0])
R[:N,[20,21]] = rs[:N]

R = R.cuda()
Amats = hmr_fusioner.smpl(root_rot=R[:,0],pose_rot=R[:,1:]).cpu().numpy()

# trajs = np.load(f'{os.path.dirname(video0_fname)}/trajs.npy').reshape((-1,1,3))
# trajs[:,:,1] += -1.1
# N = min(A.shape[0],trajs.shape[0])
# A[:N,:,0:3,3]+=trajs[:N]

Amats = np.matmul(compute_traj.camera_to_world_R, Amats)
Amats = Amats.squeeze().reshape((-1,384)).astype(np.float32)
print(Amats.shape)
export_Amatrix(f'{video0_fname[:-4]}_add_hand.txt', Amats, fps=30)

(1801, 384)


In [16]:
##step2.interpolate quats
l = 10

print('0--->', body_quats_.shape)
body_quats = body_quats_.cpu()[::l]
print('1--->',body_quats.shape)
new_quats_list = [body_quats[0]]

interp = l+1
for i in range(1,body_quats.shape[0]):
    q0 = body_quats[i-1]
    q1 = body_quats[i]
    for j in range(1,interp-1):
        a = j/interp
        q = qslerp(q0,q1,a)
        new_quats_list.append(q)
    new_quats_list.append(q1)
new_quats = torch.stack(new_quats_list)
print('2--->', new_quats.shape)

##step3.combine rot to A
R = quat2rodrigues(new_quats.reshape(-1,4)).reshape(-1,24,3,3)
R = R.cuda()
R[:,[12,15,22,23]] = torch.eye(3).reshape(1,1,3,3).cuda()
interped_A = hmr_fusioner.smpl(root_rot=R[:,0],pose_rot=R[:,1:])
interped_A = interped_A.cpu().numpy()

# trajs = np.load(f'{os.path.dirname(video0_fname)}/trajs.npy').reshape((-1,1,3))
# trajs[:,:,1] += -1.1
# N = min(interped_A.shape[0],trajs.shape[0])
# interped_A[:N,:,0:3,3]+=trajs[:N]

interped_A = np.matmul(compute_traj.camera_to_world_R,interped_A)

interped_A = interped_A.squeeze().reshape((-1,384)).astype(np.float32)
interped_A = np.nan_to_num(interped_A)

export_Amatrix(f'{video0_fname[:-4]}_new_slerp_fusion2.txt',interped_A, fps=30)

0---> torch.Size([1801, 24, 4])
1---> torch.Size([181, 24, 4])
2---> torch.Size([1801, 24, 4])


In [ ]:
window_name = "reID window"
cv2.namedWindow(window_name)
## video1
cap0 = cv2.VideoCapture(video0_fname)
h0 = int(cap0.get(cv2.CAP_PROP_FRAME_HEIGHT))
w0 = int(cap0.get(cv2.CAP_PROP_FRAME_WIDTH))
## video2
cap1 = cv2.VideoCapture(video1_fname)
h1 = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))
w1 = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))

height = h0+h1
width = max(w0 , w1)
canvas = np.zeros((height, width, 3),dtype=np.uint8)
mqtt_player = Unity3DMqttPlayer(MQTTPLAYER_CFG)

for i in range(1000):
    is_valid0,frame0 = cap0.read()
    is_valid1,frame1 = cap1.read()
    if not is_valid0 or not is_valid1:
        break
    if i in mview_x2ds_dict:
        mqtt_player([0],A[i].reshape(-1,384))
        draw_kp(frame0,mview_x2ds_dict[i][0],fmt="smpl24",color=(255,255,255))
        draw_kp(frame1,mview_x2ds_dict[i][1],fmt="smpl24",color=(255,255,255))        

    canvas[:h0] = frame0
    canvas[h0:] = frame1
    new_canvas = cv2.resize(canvas,(800,900))
    cv2.imshow(window_name,new_canvas)
    if cv2.waitKey(5) & 0xFF==ord('q'):
        break
cv2.destroyAllWindows()

In [ ]:
cv2.destroyAllWindows()

In [ ]:
##step1.interpolate traj
interp = 12
new_traj = [smoothed_traj[:1]]
for i in range(1,smoothed_traj.shape[0]):
    p0 = smoothed_traj[i-1]
    p1 = smoothed_traj[i]
    t = np.linspace(p0,p1,interp)
    new_traj.append(t[1:])
new_traj = np.concatenate(new_traj,axis=0)
print(new_traj.shape)

##step2.interpolate quats
body_quats = body_quats.cpu()
new_quats_list = [body_quats[0]]
for i in range(1,body_quats.shape[0]):
    q0,q1 = body_quats[i-1],body_quats[i]
    for j in range(1,interp-1):
        q = qslerp(q0,q1,j/interp)
        new_quats_list.append(q)
    new_quats_list.append(q1)
new_quats = torch.stack(new_quats_list)

##step3.combine rot to A
R = quat2rodrigues(new_quats.reshape(-1,4)).reshape(-1,24,3,3)
R = R.cuda()
R[:,[12,15]] = torch.eye(3).reshape(1,1,3,3).cuda()
interped_A = hmr_fusioner.smpl(root_rot=R[:,0],pose_rot=R[:,1:])
interped_A = interped_A.cpu().numpy()

interped_A[...,:3,-1] += new_traj.reshape(-1,1,3)  #update traj in A
interped_A = np.matmul(compute_traj.camera_to_world_R,interped_A)

In [ ]:
mqtt_player = Unity3DMqttPlayer(MQTTPLAYER_CFG)
for i in range(interped_A.shape[0]):
    mqtt_player([0],interped_A[i].reshape(-1,384))
    time.sleep(1/60)

In [ ]:
export_Amatrix("E:/20190727-area51/tripleSalchow_c0_20200115_002.txt",interped_A,fps=10)

In [ ]:
interped_A.shape

In [ ]:
interped_A.reshape(-1,384).shape